## Dataset Labeler Tool 
A simple interface for categorizing images into labeled folders in ipynb. Useful for quickly prepping categorical image data for deep learning / ML.  
Be sure to run each cell.

1️⃣. First create a list of the Categories to separate into.  
<span style="font-size:0.8em;">It may be helpful to include a "Trash" grouping for images that are outside of scope.</span>  

In [ ]:
categories = ["Interior", "Exterior", "Trash"]

2️⃣. Select the input and output folders

In [ ]:
import ipywidgets as widgets
import os
input = widgets.Dropdown(options=[x for x in os.listdir() if "." not in x], # folders with "." are excluded
                        description="Input folder:",)
output = widgets.Dropdown(options=[x for x in os.listdir() if "." not in x], # folders with "." are excluded
                        description="Out Folder:",)
display(widgets.VBox([input, output]))

3️⃣. Prepare output folders (creates folders for categories that are not present)

In [ ]:
for fold in [x for x in categories if x not in [x for x in os.listdir(output.value) if "." not in x]]:
    os.mkdir(output.value + "/" + fold)

4️⃣. Sort

In [ ]:
def nextpic():
    global i
    i+=1
    imgframe.value = open(input.value + "/" + piclist[i],"rb").read()
def send2folder(fold):
    os.rename(input.value + "/" + piclist[i], output.value + "/" + fold + "/" + piclist[i])
    nextpic()
def oopsgoback1():
    global i
    i-=1
    os.rename(output.value + "/" + fold + "/" + piclist[i], input.value + "/" + piclist[i])
    imgframe.value = open(input.value + "/" + piclist[i],"rb").read()

piclist = [x for x in os.listdir(input.value) if "." in x]
global i
i=0
image0 = open(input.value + "/" + piclist[0],"rb").read()
imgframe = widgets.Image(value=image0
                        , width=300 
                        # , height=300
                        )
butlist = [widgets.Button(description=cat, button_style="danger") for cat in categories]
[button.on_click(lambda x, i=cat: send2folder(i)) for button, cat in zip(butlist, categories)]

oopsbutton = widgets.Button(description="Take me back", button_style="warning")
oopsbutton.on_click(oopsgoback1())
display(widgets.HBox(butlist), imgframe, oopsbutton)